# Benchmarking Seq2Seq on Chaotic Oscillators

Seq2Seq models have been shown to be effective in modeling chaotic systems. This benchmark contains code to train and evaluate Seq2Seq models on the following chaotic systems :
  - [Lorenz](###Lorenz)
  - [Fibonacci](###Fibonacci)
  - [Van Der Pol](###Van-Der-Pol)

## Setup

In [ ]:
import os
import sys
import torch

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from src.utils.seed_utils import set_global_seeds
except ImportError:
    raise ImportError("Cannot import module. Make sure that the project is on the path")

SEED = 42
set_global_seeds(seed=SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Preprocessing


In [ ]:
BATCH_SIZE = 32
INPUT_LENGTH = 50
TARGET_LENGTH = 50

### Lorenz


In [ ]:
from src.data_sources.lorenz import LorenzOscillator
from src.utils.dataset_utils import DatasetUtils

# Example parameters
LORENZ_INPUT_LAYERS = 3
LORENZ_OUTPUT_LAYERS = 3

# Lorenz oscillator parameters
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 100)
max_step = 1e-2

lorenz = LorenzOscillator(sigma=10, rho=28, beta=8.0 / 3.0)

lorenz_dataset = lorenz.preprocess_and_create_dataset(
    initial_state=initial_state,
    t_span=t_span,
    max_step=max_step,
    input_length=INPUT_LENGTH,
    target_length=TARGET_LENGTH,
)

# Split into train/test loaders
lorenz_train_loader, lorenz_test_loader = DatasetUtils().train_test_split(
    dataset=lorenz_dataset,
    batch_size=BATCH_SIZE,
    train_ratio=0.8,
    shuffle_train=False,
    shuffle_test=False,
)

# Visualize the 3D Lorenz trajectory
lorenz.plot_trajectory(initial_state=initial_state, t_span=t_span, max_step=max_step)

In [ ]:
from src.utils.dataset_utils import DatasetUtils
from src.data_sources.emg import EMGRawDataset

root_dir = "../data/silent_speech_dataset/raw/nonparallel_data"

emg_dataset = EMGRawDataset(root_dir)

preprocessed_data = emg_dataset.load_and_preprocess_data(
    max_files=10,
)

timeseries_dataset = emg_dataset.create_dataset(
    input_length=INPUT_LENGTH, target_length=TARGET_LENGTH
)

emg_raw_train_loader, emg_raw_test_loader = DatasetUtils().train_test_split(
    dataset=timeseries_dataset,
    batch_size=32,
    train_ratio=0.8,
    shuffle_train=False,
    shuffle_test=False,
)

num_channels = preprocessed_data.shape[1]
print(f"Number of EMG channels: {num_channels}")

## Seq2Seq

In [ ]:
import os
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from src.schemas.paths_schema import PathsSchema
from src.utils.torch_utils import criterion_choice
from src.utils.tensorboard_utils import launch_tensorboard
from typing import Any


def train_seq2seq(
    model: torch.nn.Module,
    train_data: Any,
    test_data: Any,
    device: torch.device,
    SEED: int,
    BATCH_SIZE: int,
    teacher_forcing_ratio: float = 0.5,
    loss_function: str = "mse",
    learning_rate: float = 1e-6,
    num_epochs: int = 1,
):
    """
    Trains a provided Seq2Seq model using the given training and testing data loaders.
    Logs training and test losses to TensorBoard and saves the final model.
    
    Args:
        model: The Seq2Seq model instance. It must accept inputs (x, y, teacher_forcing_ratio).
        train_data: DataLoader for training data (expects batches of (x, y)).
        test_data: DataLoader for testing data (expects batches of (x, y)).
        device: Device to use for training (e.g., torch.device("cuda") or torch.device("cpu")).
        SEED: Seed used for experiment naming.
        BATCH_SIZE: Batch size used during training.
        teacher_forcing_ratio: Probability of using teacher forcing during training.
        loss_function: Name of the loss function (e.g., "mse").
        learning_rate: Learning rate for the optimizer.
        num_epochs: Number of epochs to train.
        
    Returns:
        model: The trained Seq2Seq model.
    """
    model = model.to(device)
    criterion = criterion_choice(loss_function)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Setup paths for TensorBoard logs and model saving.
    model_name = os.path.join(
        "lorenz",
        "seq2seq",
        f"seed_{SEED}_batch_size_{BATCH_SIZE}_num_epochs_{num_epochs}_lr_{learning_rate}"
    )
    paths = PathsSchema(model_name=model_name)
    writer = SummaryWriter(log_dir=paths.tensorboard_log_model)
    
    # Launch TensorBoard (optional)
    launch_tensorboard(paths.tensorboard_log_model)
    
    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Training Seq2Seq"):
        model.train()
        train_loss = 0.0
        
        for x, y in train_data:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x, y, teacher_forcing_ratio=teacher_forcing_ratio)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        avg_train_loss = train_loss / len(train_data)
        
        # Evaluation phase
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for x, y in test_data:
                x, y = x.to(device), y.to(device)
                outputs = model(x, y, teacher_forcing_ratio=0)
                loss = criterion(outputs, y.squeeze(dim=1))
                test_loss += loss.item()
        avg_test_loss = test_loss / len(test_data)
        
        writer.add_scalars("Loss", {"train": avg_train_loss, "test": avg_test_loss}, epoch)
        
        if (epoch + 1) % max(1, (num_epochs // 10)) == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")
    
    writer.close()
    
    # Save the final model.
    model_filename = (
        paths.model_path
        + f"epoch_{num_epochs}_train_loss_{avg_train_loss:.4f}_test_loss_{avg_test_loss:.4f}.pt"
    )
    torch.save(model, model_filename)
    print(f"Model saved: '{model_filename}'")

### Lorenz

In [ ]:
from src.models import seq2seq
from src.utils.plot_utils import plot_multistep_evaluation

# Hyperparameters
hidden_size = 128
num_layers = 3
dropout = 0.2
teacher_forcing_ratio = 0.5
loss_function = "mse"
learning_rate = 1e-6
num_epochs = 1000

# Model setup
encoder = seq2seq.Encoder(LORENZ_INPUT_LAYERS, hidden_size, num_layers, dropout).to(
    device
)
decoder = seq2seq.Decoder(LORENZ_OUTPUT_LAYERS, hidden_size, num_layers, dropout).to(
    device
)
model = seq2seq.Seq2Seq(encoder, decoder).to(device)
trained_model = train_seq2seq(
    model=model,
    train_data=lorenz_train_loader,
    test_data=lorenz_test_loader,
    device=device,
    SEED=SEED,
    BATCH_SIZE=BATCH_SIZE,
    teacher_forcing_ratio=teacher_forcing_ratio,
    loss_function=loss_function,
    learning_rate=learning_rate,
    num_epochs=num_epochs,
)

plot_multistep_evaluation(model, data=lorenz_test_loader)

### EMG Data

In [ ]:
from src.models import seq2seq
from src.utils.plot_utils import plot_multistep_evaluation

# Hyperparameters
hidden_size = 32
num_layers = 3
dropout = 0.2
teacher_forcing_ratio = 0.5
loss_function = "mse"
learning_rate = 1e-3
num_epochs = 10

# Model setup
encoder = seq2seq.Encoder(num_channels, hidden_size, num_layers, dropout).to(
    device
)
decoder = seq2seq.Decoder(num_channels, hidden_size, num_layers, dropout).to(
    device
)
model = seq2seq.Seq2Seq(encoder, decoder).to(device)
trained_model = train_seq2seq(
    model=model,
    train_data=emg_raw_train_loader,
    test_data=emg_raw_test_loader,
    device=device,
    SEED=SEED,
    BATCH_SIZE=BATCH_SIZE,
    teacher_forcing_ratio=teacher_forcing_ratio,
    loss_function=loss_function,
    learning_rate=learning_rate,
    num_epochs=num_epochs,
)

plot_multistep_evaluation(model, data=emg_raw_test_loader)

In [ ]:
plot_multistep_evaluation(model, data=emg_raw_test_loader, sample_idx=15)